# Tutorial 5: Event-Driven Testing

**Goal:** Validate trained policies under realistic timing constraints.

**Time:** ~15 minutes

---

## Why This Matters

You trained a policy in **synchronous mode** (Tutorial 4). But real CPS systems have:

| Real-World Factor | Training Mode | Reality |
|-------------------|---------------|---------|
| Agent tick rates | All same | Devices: 1s, Coordinators: 60s |
| Observation delay | 0ms | 50-2000ms |
| Action delay | 0ms | 100-500ms |
| Timing jitter | None | +/-10-40% |

**HERON's key insight:** You can't test this by wrapping - it requires architectural support.

### Dual Execution Modes

| Mode | Use | Implementation |
|------|-----|----------------|
| **Synchronous** | Training | `env.step(actions)` - all agents step together |
| **Event-Driven** | Testing | `env.run_event_driven()` - agents tick independently |

## The EventScheduler

HERON's `EventScheduler` is a priority-queue based discrete event simulator:

```
EventScheduler
+-- event_queue: [(t=0.0, AGENT_TICK, device_1), 
|                 (t=0.0, AGENT_TICK, device_2),
|                 (t=0.0, AGENT_TICK, coordinator),
|                 ...]
+-- current_time: 0.0
+-- handlers: {agent_id: {AGENT_TICK: handler, ACTION_EFFECT: handler, ...}}
```

Events are processed in timestamp order. When an agent ticks, it:
1. Observes (with `obs_delay`)
2. Computes action
3. Schedules action effect (with `act_delay`)
4. Schedules next tick (with `tick_interval` + optional jitter)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
from typing import Any, ClassVar, Dict, List, Optional, Sequence
from dataclasses import dataclass, field
from gymnasium.spaces import Box

# HERON imports
from heron.scheduling import EventScheduler, Event, EventType
from heron.scheduling.tick_config import TickConfig, JitterType
from heron.core.feature import FeatureProvider
from heron.core.action import Action
from heron.agents.field_agent import FieldAgent
from heron.agents.coordinator_agent import CoordinatorAgent
from heron.agents.system_agent import SystemAgent
from heron.envs.base import MultiAgentEnv

print("Imports successful!")

## Step 1: Understanding TickConfig

Each agent has a `TickConfig` that defines its timing parameters:

In [ ]:
# Deterministic config (for training - no randomness)
training_config = TickConfig.deterministic(
    tick_interval=1.0,   # Agent ticks every 1 second
    obs_delay=0.0,       # Instant observations
    act_delay=0.0,       # Instant action effects
    msg_delay=0.0,       # Instant messages
)

print("Training config (deterministic):")
print(f"  tick_interval: {training_config.get_tick_interval():.3f}s")
print(f"  obs_delay: {training_config.get_obs_delay():.3f}s")
print(f"  act_delay: {training_config.get_act_delay():.3f}s")

# Testing config with jitter (realistic timing)
testing_config = TickConfig.with_jitter(
    tick_interval=1.0,   # Base interval
    obs_delay=0.1,       # 100ms observation delay
    act_delay=0.2,       # 200ms action delay
    msg_delay=0.05,      # 50ms message delay
    jitter_type=JitterType.GAUSSIAN,  # Gaussian noise
    jitter_ratio=0.1,    # 10% standard deviation
    seed=42,             # Reproducible randomness
)

print("\nTesting config (with 10% Gaussian jitter):")
for i in range(5):
    print(f"  Sample {i+1}: tick={testing_config.get_tick_interval():.3f}s, "
          f"obs={testing_config.get_obs_delay():.3f}s, "
          f"act={testing_config.get_act_delay():.3f}s")

## Step 2: Define Agents with Timing Configuration

Each agent has timing attributes via `tick_config`:

In [ ]:
# Features (using current API with ClassVar visibility)
@dataclass(slots=True)
class BatterySOC(FeatureProvider):
    """Battery state of charge."""
    visibility: ClassVar[Sequence[str]] = ['owner', 'upper_level']
    soc: float = 0.5
    capacity: float = 2.0


@dataclass(slots=True)
class SystemFrequency(FeatureProvider):
    """System frequency."""
    visibility: ClassVar[Sequence[str]] = ['system']
    frequency_hz: float = 60.0


class TimedBattery(FieldAgent):
    """Battery with configurable timing parameters."""
    
    def __init__(
        self, 
        agent_id: str,
        tick_interval: float = 1.0,
        obs_delay: float = 0.0,
        act_delay: float = 0.0,
        use_jitter: bool = False,
        **kwargs
    ):
        self.max_power = 0.5
        
        # Create tick_config based on mode
        if use_jitter:
            tick_config = TickConfig.with_jitter(
                tick_interval=tick_interval,
                obs_delay=obs_delay,
                act_delay=act_delay,
                jitter_type=JitterType.GAUSSIAN,
                jitter_ratio=0.1,
                seed=42,
            )
        else:
            tick_config = TickConfig.deterministic(
                tick_interval=tick_interval,
                obs_delay=obs_delay,
                act_delay=act_delay,
            )
        
        features = [BatterySOC(soc=0.5, capacity=2.0)]
        super().__init__(
            agent_id=agent_id,
            features=features,
            tick_config=tick_config,
            **kwargs
        )

    def init_action(self, features: List[FeatureProvider] = []) -> Action:
        action = Action()
        action.set_specs(dim_c=1, range=(np.array([-1.0]), np.array([1.0])))
        return action

    def set_action(self, action: Any, *args, **kwargs) -> None:
        if hasattr(action, '__iter__'):
            val = action[0] if len(action) > 0 else 0.0
        else:
            val = float(action)
        self.action.set_values(val)

    def set_state(self, **kwargs) -> None:
        if 'soc' in kwargs:
            self.state.features['BatterySOC'].soc = np.clip(float(kwargs['soc']), 0.1, 0.9)

    def apply_action(self) -> None:
        soc_feature = self.state.features['BatterySOC']
        power = self.action.vector()[0] * self.max_power
        new_soc = soc_feature.soc + power / soc_feature.capacity
        self.set_state(soc=new_soc)

    def compute_local_reward(self, local_state: dict) -> float:
        soc = local_state.get('features', {}).get('BatterySOC', {}).get('soc', 0.5)
        return -abs(0.5 - soc)  # Reward for staying near 50%

    @property
    def soc(self) -> float:
        return self.state.features['BatterySOC'].soc


print("TimedBattery agent defined with configurable timing!")

In [ ]:
# Create agents with different timing configurations
# Device-level: fast updates (1s), some delay
fast_battery = TimedBattery(
    agent_id='battery_fast',
    tick_interval=1.0,
    obs_delay=0.05,
    act_delay=0.1,
    use_jitter=True,
)

# Coordinator-level: slower updates (5s), more delay
slow_battery = TimedBattery(
    agent_id='battery_slow',
    tick_interval=5.0,
    obs_delay=0.2,
    act_delay=0.5,
    use_jitter=True,
)

print(f"Fast battery tick config: {fast_battery.tick_config}")
print(f"Slow battery tick config: {slow_battery.tick_config}")

## Step 3: Environment with Event-Driven Execution

HERON's `MultiAgentEnv` provides built-in event-driven execution via `run_event_driven()`:

In [ ]:
from heron.scheduling.scheduler import EventAnalyzer, EpisodeResult


@dataclass
class SimpleEnvState:
    """Environment state."""
    battery_soc: Dict[str, float] = field(default_factory=dict)
    frequency: float = 60.0


class SimpleMicrogrid(CoordinatorAgent):
    """Simple coordinator."""
    
    def __init__(self, agent_id: str, subordinates: Dict[str, TimedBattery], **kwargs):
        super().__init__(
            agent_id=agent_id,
            subordinates=subordinates,
            tick_config=TickConfig.deterministic(tick_interval=60.0),
            **kwargs
        )


class SimpleSystem(SystemAgent):
    """Simple system agent."""
    
    def __init__(self, agent_id: str, subordinates: Dict[str, SimpleMicrogrid]):
        features = [SystemFrequency(frequency_hz=60.0)]
        super().__init__(
            agent_id=agent_id,
            features=features,
            subordinates=subordinates,
            tick_config=TickConfig.deterministic(tick_interval=300.0),
        )


class DualModeEnvironment(MultiAgentEnv):
    """Environment supporting both synchronous and event-driven modes."""
    
    def __init__(
        self,
        system_agent: SystemAgent,
        max_steps: int = 100,
    ):
        self.max_steps = max_steps
        self._step_count = 0
        
        super().__init__(
            system_agent=system_agent,
            env_id="dual_mode_env",
        )

    def global_state_to_env_state(self, global_state: Dict) -> SimpleEnvState:
        env_state = SimpleEnvState()
        agent_states = global_state.get("agent_states", {})
        for agent_id, state_dict in agent_states.items():
            features = state_dict.get("features", {})
            if "BatterySOC" in features:
                env_state.battery_soc[agent_id] = features["BatterySOC"].get("soc", 0.5)
        return env_state

    def run_simulation(self, env_state: SimpleEnvState, *args, **kwargs) -> SimpleEnvState:
        avg_soc = np.mean(list(env_state.battery_soc.values())) if env_state.battery_soc else 0.5
        env_state.frequency = 60.0 + (avg_soc - 0.5) * 0.1
        self._step_count += 1
        return env_state

    def env_state_to_global_state(self, env_state: SimpleEnvState) -> Dict:
        agent_states = self.proxy_agent.get_serialized_agent_states()
        for agent_id, state_dict in agent_states.items():
            features = state_dict.get("features", {})
            if "SystemFrequency" in features:
                features["SystemFrequency"]["frequency_hz"] = env_state.frequency
        return {"agent_states": agent_states}

    def reset(self, *, seed: Optional[int] = None, **kwargs):
        self._step_count = 0
        return super().reset(seed=seed, **kwargs)


print("DualModeEnvironment defined!")

In [ ]:
def create_dual_mode_env(use_jitter: bool = False) -> DualModeEnvironment:
    """Create environment with agents."""
    battery_1 = TimedBattery(
        agent_id='battery_1',
        tick_interval=1.0,
        obs_delay=0.05 if use_jitter else 0.0,
        act_delay=0.1 if use_jitter else 0.0,
        use_jitter=use_jitter,
        upstream_id='microgrid',
    )
    battery_2 = TimedBattery(
        agent_id='battery_2',
        tick_interval=1.0,
        obs_delay=0.05 if use_jitter else 0.0,
        act_delay=0.1 if use_jitter else 0.0,
        use_jitter=use_jitter,
        upstream_id='microgrid',
    )
    
    microgrid = SimpleMicrogrid(
        agent_id='microgrid',
        subordinates={'battery_1': battery_1, 'battery_2': battery_2},
        upstream_id='system',
    )
    
    system = SimpleSystem(
        agent_id='system',
        subordinates={'microgrid': microgrid},
    )
    
    return DualModeEnvironment(
        system_agent=system,
        max_steps=100,
    )


test_env = create_dual_mode_env(use_jitter=False)
print(f"Created environment with agents: {list(test_env.registered_agents.keys())}")

## Step 4: Event-Driven Execution with EventAnalyzer

HERON provides `run_event_driven()` for event-driven simulation with analysis:

In [ ]:
class SimpleEventAnalyzer(EventAnalyzer):
    """Analyze events during event-driven execution."""
    
    def __init__(self):
        self.events = []
    
    def parser_event(self, event: Event) -> Dict:
        """Parse and log event."""
        event_info = {
            'timestamp': event.timestamp,
            'agent_id': event.agent_id,
            'event_type': event.event_type.name,
        }
        self.events.append(event_info)
        return event_info


env = create_dual_mode_env(use_jitter=True)
obs, info = env.reset()

print("Running event-driven simulation...")

analyzer = SimpleEventAnalyzer()

result = env.run_event_driven(
    event_analyzer=analyzer,
    t_end=10.0,
    max_events=100,
)

print(f"\nProcessed {len(analyzer.events)} events")
print(f"\nFirst 10 events:")
for event in analyzer.events[:10]:
    print(f"  t={event['timestamp']:6.3f}s: {event['agent_id']} - {event['event_type']}")

In [ ]:
import matplotlib.pyplot as plt

agent_events = {}
for event in analyzer.events:
    agent_id = event['agent_id']
    if agent_id not in agent_events:
        agent_events[agent_id] = []
    if event['event_type'] == 'AGENT_TICK':
        agent_events[agent_id].append(event['timestamp'])

fig, ax = plt.subplots(figsize=(12, 4))

colors = ['blue', 'red', 'green', 'orange', 'purple']
y_pos = 1
yticks = []
ylabels = []

for i, (agent_id, ticks) in enumerate(agent_events.items()):
    if ticks:
        ax.eventplot([ticks], lineoffsets=[y_pos], linelengths=0.8,
                     colors=[colors[i % len(colors)]], linewidths=2)
        yticks.append(y_pos)
        ylabels.append(agent_id)
        y_pos += 1

ax.set_yticks(yticks)
ax.set_yticklabels(ylabels)
ax.set_xlabel('Time (s)')
ax.set_title('Agent Tick Events (Event-Driven Mode)')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

for agent_id, ticks in agent_events.items():
    print(f"{agent_id}: {len(ticks)} tick events")

## Step 5: Compare Synchronous vs Event-Driven

Let's run the same policy in both modes and compare results:

In [ ]:
def run_synchronous_episode(env, policy_fn, max_steps=50):
    """Run episode in synchronous mode."""
    obs, _ = env.reset()
    soc_history = {'battery_1': [], 'battery_2': []}
    total_reward = 0
    
    for step in range(max_steps):
        actions = {}
        for mg_id, mg in env._system_agent.subordinates.items():
            for bat_id, bat in mg.subordinates.items():
                obs_array = np.array([bat.soc], dtype=np.float32)
                actions[bat_id] = policy_fn(obs_array)
        
        obs, rewards, terminateds, truncateds, infos = env.step(actions)
        
        for mg_id, mg in env._system_agent.subordinates.items():
            for bat_id, bat in mg.subordinates.items():
                if bat_id in soc_history:
                    soc_history[bat_id].append(bat.soc)
        
        total_reward += sum(rewards.values())
        
        if terminateds.get('__all__', False):
            break
    
    return total_reward, soc_history


def simple_policy(obs):
    soc = obs[0]
    if soc < 0.45:
        return np.array([0.5])
    elif soc > 0.55:
        return np.array([-0.5])
    return np.array([0.0])


sync_env = create_dual_mode_env(use_jitter=False)
sync_reward, sync_history = run_synchronous_episode(sync_env, simple_policy, max_steps=50)
print(f"Synchronous mode: Total reward = {sync_reward:.2f}")

jitter_env = create_dual_mode_env(use_jitter=True)
jitter_reward, jitter_history = run_synchronous_episode(jitter_env, simple_policy, max_steps=50)
print(f"With timing jitter: Total reward = {jitter_reward:.2f}")

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].plot(sync_history['battery_1'], 'b-', label='battery_1')
axes[0].plot(sync_history['battery_2'], 'r-', label='battery_2')
axes[0].axhline(y=0.5, color='k', linestyle='--', alpha=0.5, label='target')
axes[0].set_xlabel('Step')
axes[0].set_ylabel('SOC')
axes[0].set_title(f'Synchronous Mode (reward={sync_reward:.1f})')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(jitter_history['battery_1'], 'b-', label='battery_1')
axes[1].plot(jitter_history['battery_2'], 'r-', label='battery_2')
axes[1].axhline(y=0.5, color='k', linestyle='--', alpha=0.5, label='target')
axes[1].set_xlabel('Step')
axes[1].set_ylabel('SOC')
axes[1].set_title(f'With Timing Jitter (reward={jitter_reward:.1f})')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nPerformance gap: {(sync_reward - jitter_reward):.2f} reward units")
print("This gap shows the policy's sensitivity to timing variations.")

## Step 6: CPS-Calibrated Timing Configurations

For realistic testing, use timing distributions from real CPS standards:

In [ ]:
scada_timing = TickConfig.with_jitter(
    tick_interval=60.0,
    obs_delay=2.0,
    act_delay=1.5,
    jitter_type=JitterType.GAUSSIAN,
    jitter_ratio=0.4,
    seed=42,
)

traffic_timing = TickConfig.with_jitter(
    tick_interval=0.1,
    obs_delay=0.05,
    act_delay=0.2,
    jitter_type=JitterType.UNIFORM,
    jitter_ratio=0.2,
    seed=42,
)

print("CPS-Calibrated Timing Configurations:")
print(f"\nSCADA (IEEE 2030):")
print(f"  tick_interval: {scada_timing.tick_interval}s")
print(f"  obs_delay: {scada_timing.obs_delay}s")
print(f"  Sample intervals: {[scada_timing.get_tick_interval() for _ in range(3)]}")

print(f"\nTraffic (NTCIP 1202):")
print(f"  tick_interval: {traffic_timing.tick_interval}s")
print(f"  obs_delay: {traffic_timing.obs_delay}s")
print(f"  Sample intervals: {[traffic_timing.get_tick_interval() for _ in range(3)]}")

## Key Takeaways

### HERON Event-Driven Features

1. **TickConfig**: Agent-level timing configuration
   ```python
   tick_config = TickConfig.with_jitter(
       tick_interval=1.0,
       obs_delay=0.1,
       act_delay=0.2,
       jitter_type=JitterType.GAUSSIAN,
       jitter_ratio=0.1,
   )
   ```

2. **EventScheduler**: Priority-queue based discrete event simulation
   ```python
   env.run_event_driven(
       event_analyzer=analyzer,
       t_end=10.0,
   )
   ```

3. **Heterogeneous Timing**: Different agents can have different tick rates

### The Testing Workflow

```
1. Train in synchronous mode (fast, deterministic)
       |
2. Test with timing jitter (realistic uncertainty)
       |
3. Test in full event-driven mode (heterogeneous timing)
       |
4. If performance degrades -> retrain with timing awareness
```

### HERON Timing Parameters

| Parameter | Meaning | Example |
|-----------|---------|--------|
| `tick_interval` | How often agent acts | Device: 1s, Coord: 60s |
| `obs_delay` | Sensor latency | 50ms - 2s |
| `act_delay` | Actuator latency | 100ms - 500ms |
| `msg_delay` | Communication latency | 10ms - 200ms |
| `jitter_ratio` | Timing variance | 10-40% |

---

**Next:** [06_configuration_and_datasets.ipynb](06_configuration_and_datasets.ipynb) - YAML configs and datasets